In [ ]:
import numpy as np
import pickle
import pandas as pd
import os

from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

Source: https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb


First, load the data set:

In [ ]:
# load the data from the csv file
df = pd.read_csv("/content/drive/MyDrive/1Jupyter/SCRIPTIE/data.csv")

# load the mlb files back in, to get the classes and transform functions
with open("/content/drive/MyDrive/1Jupyter/SCRIPTIE/dom_mlb.pkl", "rb") as f:
    dom_mlb = pickle.load(f)

with open("/content/drive/MyDrive/1Jupyter/SCRIPTIE/sub_mlb.pkl", "rb") as f:
    sub_mlb = pickle.load(f)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.1.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Next, we want to tokenize all elements in X:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
df["tokenized_text"] = df["text"].apply(lambda x: tokenizer(x, add_special_tokens=False))
print(type(df["tokenized_text"][2]))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors


<class 'transformers.tokenization_utils_base.BatchEncoding'>


After tokenizing, we need to split the text up into chunks of 512, so the sentence bert model can generate its embeddings.

In [ ]:
def chunk_text(tokens, chunk_size=512):
  if len(tokens["input_ids"]) <= chunk_size:
    return [tokenizer.decode(tokens["input_ids"])]

  chunks = []
  for i in range(0, len(tokens["input_ids"]), chunk_size):
      chunk = {k: t[i:i + chunk_size] for k, t in tokens.items()}
      chunks.append(tokenizer.decode(chunk["input_ids"]))

  return chunks

In [ ]:
df["chunked_text"] = df["tokenized_text"].apply(lambda x: chunk_text(x, 512))

The texts are successfully split into chunks with a maximum length of 512, so now let's generate the embeddings. We used an SBert model for this, but

In [ ]:
# function to generate the embeddings, takes a df with "chunked_text" and a bertmodel and returns a pd DF with the embeddings
def generate_embeddings(df, bertmodel):
  return df["chunked_text"].apply(lambda x: np.average([bertmodel.encode(i) for i in x],
                                                        weights = [len(i) for i in x],
                                                        axis = 0))

Lets generate some embeddings for a couple different SBert models:
https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [ ]:
import sys
# suppresses the loading bars that the models have, to reduce spam
original_stdout = sys.stdout
sys.stdout = open(os.devnull, 'w')
SBERTmodels = {
    "mpnet": SentenceTransformer("all-mpnet-base-v2"),
    "multiqa": SentenceTransformer("multi-qa-mpnet-base-dot-v1"),
    "distilroberta": SentenceTransformer("all-distilroberta-v1"),
    "minilm": SentenceTransformer("all-MiniLM-L12-v2"),
    "paraphrasemultilang": SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
}
sys.stdout = original_stdout

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# when training on sub categories, some simply do not have enough data in the 600 point dataset... so for now only train it on the dominant categories
def logistic_regression_classifier(X_train, y_train, X_test):
    lr = MultiOutputClassifier(LogisticRegression(class_weight="balanced", solver= "liblinear", max_iter=100))
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    return y_pred


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# this code generates the dataframe of embeddings. Per document, it encodes every chunk, and averages it into the final embedding by the number of words per chunk
f1_scores =  {}
embeddings = {}

for model_name, model in SBERTmodels.items():
  embeddings[model_name] = generate_embeddings(df, model)

  X = np.vstack(embeddings[model_name].values)
  y = df[dom_mlb.classes_].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
  y_pred = logistic_regression_classifier(X_train=X_train,y_train=y_train,X_test=X_test)

  class_report = classification_report(y_true=y_test,y_pred=y_pred,target_names=dom_mlb.classes_, output_dict=True, zero_division=0.0)

  print(f"{model_name} embeddings:")
  print(classification_report(y_true=y_test,y_pred=y_pred,target_names=dom_mlb.classes_, zero_division=0.0))
  print("----------------------------------------------------------------------------------")

  f1_scores[model_name] = class_report["samples avg"]['f1-score']

mpnet embeddings:
                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.54      1.00      0.70         7
                      CC: Climate change is beneficial       0.00      0.00      0.00         0
              CC: Controversy about green technologies       0.20      1.00      0.33         1
                     CC: Criticism of climate movement       0.44      1.00      0.62         4
                     CC: Criticism of climate policies       0.40      0.80      0.53         5
         CC: Criticism of institutions and authorities       0.40      1.00      0.57         6
                        CC: Downplaying climate change       0.22      1.00      0.36         2
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         0
 CC: Hidden plots by secret schemes of powerful groups       0.29      0.67      0.40         3
          CC: Questio

Not amazing, but at least we now know the embedding is working as it should!

Lastly, lets save the best embeddings now that we have them. The embeddings are saved as a pickle file. to do this, we take the max of the f1 sample scores, for which we want to optimize the task.

In [ ]:
bestembeddings = embeddings[max(f1_scores)]
print(bestembeddings)

with open("/content/drive/MyDrive/1Jupyter/SCRIPTIE/embeddings.pkl", "wb") as f:
    pickle.dump(bestembeddings, f)

0      [0.15851294905673272, -0.1362919395903227, -0....
1      [-0.02812150865793228, 0.1337200105190277, -0....
2      [0.0011932615172948427, -0.039664918359867324,...
3      [0.07796116809005393, -0.19448494981831407, -0...
4      [-0.09016437083482742, -0.11372177302837372, -...
                             ...                        
621    [0.06095129996538162, -0.06814509630203247, -0...
622    [-0.061890374807230496, 0.07734311149534533, -...
623    [-0.04900439828634262, -0.10150399804115295, -...
624    [0.017338981528675284, 0.07199527126815737, -0...
625    [0.04894140362739563, -0.12287387251853943, -0...
Name: chunked_text, Length: 626, dtype: object


Use the following code to open up the embedding file

In [ ]:
with open("/content/drive/MyDrive/1Jupyter/SCRIPTIE/embeddings.pkl", "rb") as f:
    bestembeddings = pickle.load(f)

print(bestembeddings)

0      [0.15851294905673272, -0.1362919395903227, -0....
1      [-0.02812150865793228, 0.1337200105190277, -0....
2      [0.0011932615172948427, -0.039664918359867324,...
3      [0.07796116809005393, -0.19448494981831407, -0...
4      [-0.09016437083482742, -0.11372177302837372, -...
                             ...                        
621    [0.06095129996538162, -0.06814509630203247, -0...
622    [-0.061890374807230496, 0.07734311149534533, -...
623    [-0.04900439828634262, -0.10150399804115295, -...
624    [0.017338981528675284, 0.07199527126815737, -0...
625    [0.04894140362739563, -0.12287387251853943, -0...
Name: chunked_text, Length: 626, dtype: object
